In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchsummary import summary
from torch.utils.data import DataLoader
from torchvision import datasets
import numpy as np
import random
from model_structure import get_preprocessing_transforms,BCNN, train_model, evaluate_model, get_resnet_model, get_squ_model

In [2]:
# seed = 441
# torch.manual_seed(seed)
# torch.cuda.manual_seed(seed)
# torch.cuda.manual_seed_all(seed)
# np.random.seed(seed)
# random.seed(seed)
# torch.backends.cudnn.deterministic = True
# torch.backends.cudnn.benchmark = False

In [3]:
model_number = 2

In [4]:
BATCH_SIZE = 32
LEARNING_RATE = 0.001
NUM_EPOCHS = 10
INPUT_SIZE = 224
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [5]:
# Get transforms
train_transform, val_transform = get_preprocessing_transforms(INPUT_SIZE)

In [6]:
# Set data set directory
train_dir = f'data4model_{model_number}/train/'
val_dir = f'data4model_{model_number}/test/'

In [7]:
train_dataset = datasets.ImageFolder(
        root=train_dir,
        transform=train_transform
    )
    
val_dataset = datasets.ImageFolder(
        root=val_dir,
        transform=val_transform
    )

In [8]:
# Create data loaders
train_loader = DataLoader(
    train_dataset, 
    batch_size=BATCH_SIZE, 
    shuffle=True, 
    num_workers=4,
    pin_memory=True
)

val_loader = DataLoader(
    val_dataset, 
    batch_size=BATCH_SIZE, 
    shuffle=False, 
    num_workers=4,
    pin_memory=True
)

In [15]:
model = BCNN(3).to(DEVICE)
criterion =  nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)

In [16]:
model.eval()

BCNN(
  (features): Sequential(
    (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Dropout2d(p=0.25, inplace=False)
    (4): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (5): ReLU(inplace=True)
    (6): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (7): Dropout2d(p=0.25, inplace=False)
    (8): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace=True)
    (10): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (11): Dropout2d(p=0.25, inplace=False)
  )
  (classifier): Sequential(
    (0): Flatten(start_dim=1, end_dim=-1)
    (1): Linear(in_features=100352, out_features=512, bias=True)
    (2): ReLU(inplace=True)
    (3): Dropout(p=0.5, inplace=False)
    (4): Linear(in_features=512, out_features=2, bias=True)
  )
)

In [17]:
summary(model, (3, INPUT_SIZE, INPUT_SIZE))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 32, 224, 224]             896
              ReLU-2         [-1, 32, 224, 224]               0
         MaxPool2d-3         [-1, 32, 112, 112]               0
         Dropout2d-4         [-1, 32, 112, 112]               0
            Conv2d-5         [-1, 64, 112, 112]          18,496
              ReLU-6         [-1, 64, 112, 112]               0
         MaxPool2d-7           [-1, 64, 56, 56]               0
         Dropout2d-8           [-1, 64, 56, 56]               0
            Conv2d-9          [-1, 128, 56, 56]          73,856
             ReLU-10          [-1, 128, 56, 56]               0
        MaxPool2d-11          [-1, 128, 28, 28]               0
        Dropout2d-12          [-1, 128, 28, 28]               0
          Flatten-13               [-1, 100352]               0
           Linear-14                  [

In [22]:
tracker = train_model(model, train_loader, val_loader, criterion, optimizer, NUM_EPOCHS, DEVICE, model_number)

Epoch [1/10]
Training Loss: 0.2350, Validation Loss: 0.1546
Training Acc: 0.9137, Validation Acc: 0.9350
Training F1: 0.9141, Validation F1: 0.9353
Epoch [2/10]
Training Loss: 0.1437, Validation Loss: 0.2803
Training Acc: 0.9477, Validation Acc: 0.8800
Training F1: 0.9478, Validation F1: 0.8899
Epoch [3/10]
Training Loss: 0.1305, Validation Loss: 0.5504
Training Acc: 0.9527, Validation Acc: 0.8500
Training F1: 0.9529, Validation F1: 0.8235
Epoch [4/10]
Training Loss: 0.1091, Validation Loss: 0.0992
Training Acc: 0.9600, Validation Acc: 0.9650
Training F1: 0.9599, Validation F1: 0.9645
Epoch [5/10]
Training Loss: 0.0967, Validation Loss: 0.4401
Training Acc: 0.9653, Validation Acc: 0.8100
Training F1: 0.9654, Validation F1: 0.7654
Epoch [6/10]
Training Loss: 0.0680, Validation Loss: 0.1297
Training Acc: 0.9750, Validation Acc: 0.9450
Training F1: 0.9750, Validation F1: 0.9453
Epoch [7/10]
Training Loss: 0.0999, Validation Loss: 0.1375
Training Acc: 0.9587, Validation Acc: 0.9450
Trainin